### Import required libraries

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd

# Outputs
from tabulate import tabulate
import plotly.express as px

# Linear Programming
from ortools.linear_solver import pywraplp

### Create SCIP solver

In [ ]:
# Create the solver with the GLOP solver
solver = pywraplp.Solver.CreateSolver('SCIP')

### Parameters

In [ ]:
# Single value parameters
pmax = 500  # m3/s
qmax = 200  # m3/s
vmax = 200  # hm3
v0 = 80  # hm3
C = 0.0864  # m3/s to hm3/day
days = 10  # number of time steps
inflow_scaling_factor = 25  # inflow scaling factor to create interesting model
nominal_revenue = 0.15  # CHF/kWh

# Water energy transformation
outflow_to_energy = 6 * 24  # Power (W) to Energy en kWh/day

# Solar panel parameters
solar_constant = 1.37  #kW/m2
sunlight_hours = 8  #hours / 1 day
solar_to_elect = 0.20  # kW of sun to kW of electricity
sunlight_constant = solar_constant * sunlight_hours  # kWh/m2/day
solar_panel_area = 12  #m2 of solar panels PARAMETER

max_grid = 250  #kWh
elec_to_water = 0.60


# Vector parameters
revenue_per_kwh = np.array([1.00, 0.05, 0.1, 0.1, 0.1, 0.25, 0.80, 2.00, 2.00,
                            1.00]) * nominal_revenue  # created, not from height, revenue per unit of energy (1 being baseline (100%) normal price, 2 being double price, 0.5 being half price)
## Some days, it is more profitable to store water than to sell it
a = np.array([1, 2, 16, 16, 8, 2, 8, 4, 16, 1]) * inflow_scaling_factor
sun_pred = np.array([1, 0.8, 1.2, 1.5, 0.1, 0.2, 0.3, 1, 0.9,
                     0.8])  # predicted sunlight for each day as percentage of nominal sunlight (solar_constant)


# Empty array to fill in with pumped water
pumped_water = np.array([])

### Variables

In [ ]:
# External variables
p = {}  # p[t] is the flow through the turbines at time t
q = {}  # q[t] is the flow dumped in the lake at time t

# Implicit variables
v = {}  # v[t] is the volume in reservoir at time t, hm3
prod = {}  # prod[t] is the energy production * revenue per energy unit

# Create variables, N+1 to accommodate for v[t+1] term
for t in range(days + 1):
    p[t] = solver.NumVar(0, solver.infinity(), 'p[%i]' % t)
    q[t] = solver.NumVar(0, solver.infinity(), 'q[%i]' % t)
    v[t] = solver.NumVar(0, solver.infinity(), 'v[%i]' % t)
    prod[t] = solver.NumVar(0, solver.infinity(), 'prod[%i]' % t)

print('Number of variables =', solver.NumVariables())

In [ ]:
for t in range(days):
    # Initial conditions
    solver.Add(v[0] == v0)  # initial volume

    # Calculate electricity produced from solar panels
    sol_pow = sunlight_constant * sun_pred[t] * solar_panel_area * solar_to_elect
    # kWh/m2/day * unitless * m2 * kWh/m2 = kWh/day

    # Check if the electricity produced is higher than the maximum supported from the grid
    if sol_pow > max_grid:
        # If it is higher, calculate the surplus
        elec_surplus = sol_pow - max_grid
    else:
        # # If it is lower, set the surplus to zero
        elec_surplus = 0

    # Calculate the water that could be pumped into the dam (Converting kWh/day to hm3/day) https://www.quora.com/How-many-kW-is-needed-to-pump-1000-m-3-h-of-water-to-10-m-high
    ## Volume pumped from energy amount
    ## We need to convert kWh/day surplus to hm3/day added water
    additional_water = elec_surplus * elec_to_water

    # Append the value in a numpy array
    pumped_water = np.append(pumped_water, np.int32(additional_water))

    # Constraints that contain [t-1] terms
    solver.Add(v[t + 1] == v[t] + pumped_water[t] + C * (a[t] - p[t] - q[t]))  # volume balance
    solver.Add(p[t] - p[t + 1] <= 0.5 * pmax)
    solver.Add(p[t + 1] - p[t] <= 0.5 * pmax)

    # Production from efficiency and flow in turbine
    solver.Add(prod[t] == revenue_per_kwh[t] * (p[t] * outflow_to_energy))  #p[t]* outflow_to_energy = kwh/day

    # Ceiling constraints
    solver.Add(v[t] <= vmax)
    solver.Add(p[t] <= pmax)
    solver.Add(q[t] <= qmax)

    # Non-negativity constraints
    solver.Add(v[t] >= 0)
    solver.Add(p[t] >= 0)
    solver.Add(q[t] >= 0)
    solver.Add(prod[t] >= 0)

In [ ]:
# Objective function
objective = solver.Objective()

# Equilvalent to sum(prod[t] for t in range(N)) or prod[0] + ... + prod[days - 1]
for t in range(days):
    objective.SetCoefficient(prod[t], 1)
objective.SetMaximization()

In [ ]:
# Solve
status = solver.Solve()

In [ ]:
# Extract solution
p_sol = []
q_sol = []
v_sol = []
prod_sol = []
print('Objective value =', solver.Objective().Value())
# If optimality is reached, print the solution
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for t in range(days + 1):
        print(p[t].name(), ' = ', p[t].solution_value())
        print(q[t].name(), ' = ', q[t].solution_value())
        print(v[t].name(), ' = ', v[t].solution_value())
        p_sol.append(p[t].solution_value())
        q_sol.append(q[t].solution_value())
        v_sol.append(v[t].solution_value())
        prod_sol.append(prod[t].solution_value())
        print()

In [ ]:
df = pd.DataFrame(data=[p_sol, q_sol, a, v_sol, prod_sol, revenue_per_kwh, pumped_water]).T
df.columns = ['p_t', 'q_t', 'a_t', 'v_t', 'prod_t', 'eff', "pumped_water"]
df = df.round(2)
#df.style.format(precision = 2)
print('Volume at the end of last day =', v_sol[-1])
df = df[:-1]
print(tabulate(df, headers=df.columns))
# graph all but last row on plotly
fig = px.line(df, x=df.index, y=df.columns)
fig.show()